# Perineural Immune Landscape & External Validation

**PeriNeuroImmuneMap Analysis Pipeline**

---

**Correspondence:** Wei Qin (<wqin@sjtu.edu.cn>)


In [ ]:
from pathlib import Path

BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIGURE_DIR = OUTPUT_DIR / 'figures'
TABLE_DIR  = OUTPUT_DIR / 'tables'

for d in [DATA_DIR, OUTPUT_DIR, FIGURE_DIR, TABLE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print('✓ Paths configured')


In [ ]:
# NOTEBOOK 4: NERVE-IMMUNE-TUMOR COMMUNICATION NETWORKS 

import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import json

# Plotting setup
plt.rcParams['figure.dpi'] = 1200
plt.rcParams['savefig.dpi'] = 1200
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 8

print("="*80)
print("NOTEBOOK 4: NERVE-IMMUNE-TUMOR COMMUNICATION NETWORKS")
print("="*80)
print(f"\nStart time: {datetime.now()}")
print(f"Purpose: Map nerve-immune-tumor signaling for special issue")
print(f"Expected runtime: 4-6 hours\n")

# SECTION 0: SETUP & CONFIGURATION

print("="*80)
print("SECTION 0: SETUP & CONFIGURATION")
print("="*80)

# Paths
BASE_DIR = Path(r"D:/个人文件夹/Sanwal/Neuro")
NB4_DIR = BASE_DIR / "processed/notebook4"
OUTPUT_DIR = NB4_DIR / "outputs"
FIG_DIR = OUTPUT_DIR / "figures"
TABLE_DIR = OUTPUT_DIR / "tables"

for d in [OUTPUT_DIR, FIG_DIR, TABLE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"\nDirectories configured:")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Figures: {FIG_DIR}")
print(f"  Tables: {TABLE_DIR}")

# Load data paths
print(f"\nLoading data inventory...")

# Auto-discover TCGA files
RAW_DATA = BASE_DIR / "Raw data"
TCGA_DIR = RAW_DATA / "TCGA RNA"

TCGA_EXPRESSION = {}
TCGA_CLINICAL = {}

cancer_mappings = {
    'BLCA': 'Bladder', 'BRCA': 'Breast', 'COAD': 'Colon', 'GBM': 'Glioblastoma',
    'HNSC': 'Head_Neck', 'KIRC': 'Kidney', 'LIHC': 'Liver', 'LUAD': 'Lung_Adenocarcinoma',
    'OV': 'Ovarian', 'PAAD': 'Pancreatic', 'PRAD': 'Prostate', 'READ': 'Rectal',
    'SKCM': 'Melanoma', 'STAD': 'Stomach', 'UCEC': 'Endometrial'
}

# Scan for files
for cancer_code, cancer_name in cancer_mappings.items():
    possible_folders = list(TCGA_DIR.glob(f"*{cancer_code}*")) + list(TCGA_DIR.glob(f"*{cancer_name}*"))
    
    if possible_folders:
        folder = possible_folders[0]
        
        # Expression files
        expr_files = (list(folder.glob("*expression*.txt")) + list(folder.glob("*expression*.tsv")) +
                     list(folder.glob("*fpkm*.txt")) + list(folder.glob("*tpm*.txt")) +
                     list(folder.glob("*.txt")) + list(folder.glob("*.tsv")))
        
        if expr_files:
            TCGA_EXPRESSION[cancer_code] = expr_files[0]
        
        # Clinical files
        clin_files = (list(folder.glob("*clinical*.txt")) + list(folder.glob("*clinical*.tsv")) +
                     list(folder.glob("*phenotype*.txt")))
        
        if len(expr_files) > 1 and not clin_files:  # Second file might be clinical
            TCGA_CLINICAL[cancer_code] = expr_files[1]
        elif clin_files:
            TCGA_CLINICAL[cancer_code] = clin_files[0]

THORSSON_FILE = NB4_DIR / "mmc2.xlsx"
SIGNATURE_FILE = BASE_DIR / "processed/notebook2/nerve_injury_signature_v1.0_FINAL.csv"

print(f"  [OK] Auto-discovered TCGA data")
print(f"  TCGA Expression: {len(TCGA_EXPRESSION)} cancer types")
print(f"  TCGA Clinical: {len(TCGA_CLINICAL)} cancer types")

# Verify files
if not SIGNATURE_FILE.exists():
    print(f"[ERROR] Signature file not found: {SIGNATURE_FILE}")
    sys.exit(1)
if not THORSSON_FILE.exists():
    print(f"[ERROR] Thorsson file not found: {THORSSON_FILE}")
    sys.exit(1)

print(f"  [OK] All critical files present")

# SECTION 1: LOAD NERVE INJURY SIGNATURE

print("\n" + "="*80)
print("SECTION 1: LOAD NERVE INJURY SIGNATURE")
print("="*80)

signature = pd.read_csv(SIGNATURE_FILE)
print(f"\n[OK] Loaded: {signature.shape}")

SIGNATURE_GENES = signature['gene_symbol'].dropna().tolist()
SIGNATURE_WEIGHTS = dict(zip(signature['gene_symbol'], signature['log2FC']))

print(f"  Genes: {len(SIGNATURE_GENES)}")
print(f"  Weight range: [{min(SIGNATURE_WEIGHTS.values()):.2f}, {max(SIGNATURE_WEIGHTS.values()):.2f}]")

# SECTION 2: DEFINE CURATED GENE LISTS

print("\n" + "="*80)
print("SECTION 2: CURATED GENE LISTS")
print("="*80)

# Neuropeptides (Nerve signaling)
NEUROPEPTIDES = ['NGF', 'BDNF', 'GDNF', 'TAC1', 'CALCA', 'NPY', 'VIP', 'SST']

# Cytokines (Immune signaling)
CYTOKINES = ['IL1B', 'IL6', 'TNF', 'IFNG', 'IL10', 'TGFB1', 'CCL2', 'CXCL12']

# Receptors
RECEPTORS = ['NTRK1', 'NTRK2', 'TACR1', 'CALCRL', 'IL1R1', 'IL6R', 'TNFRSF1A', 'TGFBR1']

# Ligand-Receptor pairs
LR_PAIRS = pd.DataFrame({
    'ligand': ['NGF', 'BDNF', 'TAC1', 'CALCA', 'IL1B', 'IL6', 'TNF', 'TGFB1'],
    'receptor': ['NTRK1', 'NTRK2', 'TACR1', 'CALCRL', 'IL1R1', 'IL6R', 'TNFRSF1A', 'TGFBR1'],
    'pathway': ['Neurotrophin', 'Neurotrophin', 'Substance P', 'CGRP', 
                'Inflammatory', 'Inflammatory', 'Inflammatory', 'TGF-beta'],
    'direction': ['Nerve->Immune']*4 + ['Immune->Nerve']*4
})

print(f"\n[OK] Gene lists:")
print(f"  Neuropeptides: {len(NEUROPEPTIDES)}")
print(f"  Cytokines: {len(CYTOKINES)}")
print(f"  Receptors: {len(RECEPTORS)}")
print(f"  LR pairs: {len(LR_PAIRS)}")

# Neurophysiological outcomes
NEURO_OUTCOMES = {
    'pain': ['OPRM1', 'TAC1', 'CALCA', 'TRPV1'],
    'fatigue': ['IL1B', 'IL6', 'TNF'],
    'depression': ['HTR2A', 'SLC6A4'],
    'cognition': ['DRD2', 'BDNF']
}

print(f"\nOutcome markers:")
for outcome, genes in NEURO_OUTCOMES.items():
    print(f"  {outcome.capitalize()}: {len(genes)} genes")

# SECTION 3: HELPER FUNCTIONS

print("\n" + "="*80)
print("SECTION 3: HELPER FUNCTIONS")
print("="*80)

def compute_signature_score(expr_df, sig_genes, sig_weights):
    """Compute weighted signature score"""
    overlap = list(set(expr_df.index) & set(sig_genes))
    
    if len(overlap) == 0:
        return pd.Series(index=expr_df.columns, data=np.nan)
    
    coverage = 100 * len(overlap) / len(sig_genes)
    print(f"    Coverage: {len(overlap)}/{len(sig_genes)} ({coverage:.1f}%)")
    
    expr_subset = expr_df.loc[overlap]
    weights = np.array([sig_weights[g] for g in overlap])
    
    scores = (expr_subset.T * weights).sum(axis=1) / len(overlap)
    scores = (scores - scores.mean()) / (scores.std() + 1e-8)
    
    return scores

def calculate_correlation_network(expr_df, gene_list, threshold=0.3):
    """Calculate gene co-expression network"""
    # Subset to genes
    overlap = list(set(expr_df.index) & set(gene_list))
    
    if len(overlap) < 3:
        return None, None
    
    expr_subset = expr_df.loc[overlap]
    
    # Correlations
    corr_matrix = expr_subset.T.corr()
    
    # Threshold
    corr_matrix[abs(corr_matrix) < threshold] = 0
    np.fill_diagonal(corr_matrix.values, 0)
    
    return corr_matrix, overlap

print(f"\n[OK] Functions defined")

# PART 1: TCGA PAN-CANCER EXPLORATORY ANALYSIS

print("\n" + "="*80)
print("="*80)
print("PART 1: TCGA PAN-CANCER EXPLORATORY ANALYSIS")
print("="*80)
print("="*80)

print("""
[CRITICAL FRAMING]
This section performs an exploratory analysis of nerve injury signature scores)

# SECTION 4: LOAD TCGA DATA

print("\n" + "="*80)
print("SECTION 4: LOAD TCGA PAN-CANCER DATA")
print("="*80)

print(f"\nThis will load TCGA data for 15 cancer types...")
print(f"Expected time: 5-15 minutes depending on file sizes\n")

# Initialize storage
tcga_expr_all = {}
tcga_clinical_all = {}
tcga_scores = {}

# Load each cancer type
for cancer_type in sorted(TCGA_EXPRESSION.keys()):
    print(f"\nLoading {cancer_type}...")
    
    try:
        # Expression
        expr_file = TCGA_EXPRESSION[cancer_type]
        print(f"  Reading: {expr_file.name}")
        
        # Try different formats
        if expr_file.suffix == '.txt' or expr_file.suffix == '.tsv':
            expr = pd.read_csv(expr_file, sep='\t', index_col=0, low_memory=False)
        else:
            expr = pd.read_csv(expr_file, index_col=0, low_memory=False)
        
        print(f"  [OK] Expression: {expr.shape}")
        
        # Clinical
        if cancer_type in TCGA_CLINICAL:
            clin_file = TCGA_CLINICAL[cancer_type]
            print(f"  Reading: {clin_file.name}")
            
            if clin_file.suffix == '.txt' or clin_file.suffix == '.tsv':
                clinical = pd.read_csv(clin_file, sep='\t', low_memory=False)
            else:
                clinical = pd.read_csv(clin_file, low_memory=False)
            
            print(f"  [OK] Clinical: {clinical.shape}")
        else:
            clinical = None
            print(f"  [!] No clinical data")
        
        # Store
        tcga_expr_all[cancer_type] = expr
        tcga_clinical_all[cancer_type] = clinical
        
        # Compute signature scores
        print(f"  Computing nerve scores...")
        scores = compute_signature_score(expr, SIGNATURE_GENES, SIGNATURE_WEIGHTS)
        tcga_scores[cancer_type] = scores
        
        print(f"  [OK] Scores: mean={scores.mean():.3f}, std={scores.std():.3f}")
        
    except Exception as e:
        print(f"  [ERROR] {e}")
        continue

print(f"\n[OK] Loaded {len(tcga_expr_all)} cancer types")

# SECTION 5: LOAD THORSSON IMMUNE DATA

print("\n" + "="*80)
print("SECTION 5: LOAD THORSSON IMMUNE LANDSCAPE")
print("="*80)

print(f"\nLoading: {THORSSON_FILE.name}")

thorsson = pd.read_excel(THORSSON_FILE, sheet_name=0)
print(f"[OK] Loaded: {thorsson.shape}")

print(f"\nColumns (first 10):")
for i, col in enumerate(list(thorsson.columns)[:10], 1):
    print(f"  {i:2d}. {col}")

# Extract key columns
thorsson_subset = thorsson[[
    'TCGA Participant Barcode',
    'TCGA Study',
    'Immune Subtype',
    'Leukocyte Fraction',
    'Stromal Fraction'
]].copy()

print(f"\n[OK] Extracted immune phenotypes")
print(f"  Samples: {len(thorsson_subset)}")
print(f"  Immune subtypes: {thorsson_subset['Immune Subtype'].nunique()}")

# SECTION 6: COMPUTE PAN-CANCER STATISTICS

print("\n" + "="*80)
print("SECTION 6: PAN-CANCER NERVE SCORE STATISTICS")
print("="*80)

# Compile statistics
stats_list = []

for cancer_type in sorted(tcga_scores.keys()):
    scores = tcga_scores[cancer_type]
    
    stats_dict = {
        'cancer_type': cancer_type,
        'n_samples': len(scores),
        'mean_score': scores.mean(),
        'std_score': scores.std(),
        'median_score': scores.median(),
        'q25': scores.quantile(0.25),
        'q75': scores.quantile(0.75),
        'min_score': scores.min(),
        'max_score': scores.max()
    }
    
    stats_list.append(stats_dict)

stats_df = pd.DataFrame(stats_list)
stats_df = stats_df.sort_values('mean_score', ascending=False)

print(f"\n[OK] Computed statistics for {len(stats_df)} cancer types")
print(f"\nTop 5 cancer types by mean nerve score:")
print(stats_df[['cancer_type', 'n_samples', 'mean_score']].head())

# Save
stats_file = TABLE_DIR / "Table_S1_TCGA_Nerve_Scores_Summary.csv"
stats_df.to_csv(stats_file, index=False)
print(f"\n[OK] Saved: {stats_file.name}")

# SECTION 7: ASSOCIATE WITH IMMUNE PHENOTYPES

print("\n" + "="*80)
print("SECTION 7: NERVE SCORES vs IMMUNE PHENOTYPES")
print("="*80)

print(f"\nAnalyzing association with Thorsson immune subtypes...")

# Merge nerve scores with immune data
immune_nerve_data = []

for cancer_type, scores in tcga_scores.items():
    # Get samples for this cancer
    cancer_thorsson = thorsson_subset[thorsson_subset['TCGA Study'].str.contains(cancer_type, case=False, na=False)]
    
    if len(cancer_thorsson) == 0:
        continue
    
    # Match samples
    for barcode in cancer_thorsson['TCGA Participant Barcode']:
        # Try to find matching sample in scores
        matching = [s for s in scores.index if barcode in s]
        
        if matching:
            sample_id = matching[0]
            score = scores[sample_id]
            
            sample_info = cancer_thorsson[cancer_thorsson['TCGA Participant Barcode']==barcode].iloc[0]
            
            immune_nerve_data.append({
                'cancer_type': cancer_type,
                'sample_id': sample_id,
                'barcode': barcode,
                'nerve_score': score,
                'immune_subtype': sample_info['Immune Subtype'],
                'leukocyte_fraction': sample_info['Leukocyte Fraction'],
                'stromal_fraction': sample_info['Stromal Fraction']
            })

immune_nerve_df = pd.DataFrame(immune_nerve_data)
print(f"\n[OK] Matched {len(immune_nerve_df)} samples with immune data")

# Compute correlation with leukocyte fraction
if len(immune_nerve_df) > 0:
    # Drop rows with NaN in either column
    valid_data = immune_nerve_df[['nerve_score', 'leukocyte_fraction']].dropna()
    
    if len(valid_data) > 10:
        corr, pval = stats.spearmanr(
            valid_data['nerve_score'],
            valid_data['leukocyte_fraction']
        )
        
        print(f"\nNerve score vs Leukocyte fraction:")
        print(f"  Valid samples: {len(valid_data)}")
        print(f"  Spearman r = {corr:.3f}, p = {pval:.2e}")
    else:
        print(f"\n[!] Insufficient valid data for correlation")

# By immune subtype
if 'immune_subtype' in immune_nerve_df.columns:
    subtype_stats = immune_nerve_df.groupby('immune_subtype')['nerve_score'].agg(['mean', 'std', 'count'])
    subtype_stats = subtype_stats.sort_values('mean', ascending=False)
    
    print(f"\nNerve score by immune subtype:")
    print(subtype_stats)

# Save
immune_file = TABLE_DIR / "Table_S2_Nerve_Scores_Immune_Phenotypes.csv"
immune_nerve_df.to_csv(immune_file, index=False)
print(f"\n[OK] Saved: {immune_file.name}")

# SECTION 8: CREATE FIGURE 6A - PAN-CANCER PREVALENCE

print("\n" + "="*80)
print("SECTION 8: CREATE FIGURE 6A - PAN-CANCER NERVE SCORE DISTRIBUTION")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Panel A: Violin plot by cancer type
ax = axes[0, 0]
cancer_types_sorted = stats_df['cancer_type'].tolist()

# Prepare data for violin
violin_data = [tcga_scores[ct].dropna() for ct in cancer_types_sorted]

parts = ax.violinplot(violin_data, 
                     positions=range(len(cancer_types_sorted)),
                     widths=0.7,
                     showmeans=True,
                     showmedians=True)

for pc in parts['bodies']:
    pc.set_facecolor('#8dd3c7')
    pc.set_alpha(0.7)

ax.set_xticks(range(len(cancer_types_sorted)))
ax.set_xticklabels(cancer_types_sorted, rotation=45, ha='right')
ax.set_ylabel('Nerve Injury Score (Z-score)', fontsize=10)
ax.set_title('A. Nerve Score Distribution by Cancer Type', fontsize=11, fontweight='bold')
ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.5)
ax.grid(axis='y', alpha=0.3)

# Panel B: Bar plot of mean scores
ax = axes[0, 1]
x_pos = np.arange(len(cancer_types_sorted))
means = stats_df['mean_score'].values
stds = stats_df['std_score'].values

bars = ax.bar(x_pos, means, yerr=stds, 
              color='#80b1d3', alpha=0.8,
              capsize=3, error_kw={'linewidth':1})

ax.set_xticks(x_pos)
ax.set_xticklabels(cancer_types_sorted, rotation=45, ha='right')
ax.set_ylabel('Mean Nerve Score ± SD', fontsize=10)
ax.set_title('B. Mean Nerve Scores by Cancer Type', fontsize=11, fontweight='bold')
ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.5)
ax.grid(axis='y', alpha=0.3)

# Panel C: Sample sizes
ax = axes[1, 0]
samples = stats_df['n_samples'].values

bars = ax.bar(x_pos, samples, color='#fb8072', alpha=0.8)

ax.set_xticks(x_pos)
ax.set_xticklabels(cancer_types_sorted, rotation=45, ha='right')
ax.set_ylabel('Number of Samples', fontsize=10)
ax.set_title('C. Sample Sizes per Cancer Type', fontsize=11, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Panel D: Correlation with immune infiltration
ax = axes[1, 1]

if len(immune_nerve_df) > 0:
    # Get valid data (drop NaN together)
    valid_data = immune_nerve_df[['leukocyte_fraction', 'nerve_score']].dropna()
    
    if len(valid_data) > 10:
        scatter = ax.scatter(valid_data['leukocyte_fraction'],
                            valid_data['nerve_score'],
                            alpha=0.3, s=10, c='#bebada')
        
        # Regression line
        x = valid_data['leukocyte_fraction'].values
        y = valid_data['nerve_score'].values
        
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        x_line = np.linspace(x.min(), x.max(), 100)
        ax.plot(x_line, p(x_line), "r-", linewidth=2, alpha=0.8)
        
        corr, pval = stats.spearmanr(x, y)
        ax.text(0.05, 0.95, f'r = {corr:.3f}\np = {pval:.2e}',
               transform=ax.transAxes, fontsize=9,
               verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel('Leukocyte Fraction', fontsize=10)
    ax.set_ylabel('Nerve Injury Score', fontsize=10)
    ax.set_title('D. Nerve Score vs Immune Infiltration', fontsize=11, fontweight='bold')
    ax.grid(alpha=0.3)

plt.suptitle('Figure 6A: TCGA Pan-Cancer Nerve Injury Signature (EXPLORATORY)',
            fontsize=13, fontweight='bold', y=0.995)

plt.tight_layout()

fig_file = FIG_DIR / "Figure_6A_PanCancer_Nerve_Scores.pdf"
plt.savefig(fig_file, dpi=1200, bbox_inches='tight')
plt.savefig(fig_file.with_suffix('.png'), dpi=1200, bbox_inches='tight')
plt.close()

print(f"\n[OK] Saved: {fig_file.name}")

# SECTION 9: EXPLORATORY FRAMING STATEMENT

print("\n" + "="*80)
print("SECTION 9: EXPORT EXPLORATORY ANALYSIS SUMMARY")
print("="*80)

exploratory_summary = f

summary_file = OUTPUT_DIR / "00_TCGA_Exploratory_Analysis_Summary.txt"
with open(summary_file, 'w', encoding='utf-8') as f:
    f.write(exploratory_summary)

print(exploratory_summary)
print(f"\n[OK] Saved: {summary_file.name}")

# PART 2: LIGAND-RECEPTOR NETWORK INFERENCE

print("\n" + "="*80)
print("="*80)
print("PART 2: LIGAND-RECEPTOR CO-EXPRESSION NETWORKS")
print("="*80)
print("="*80)

print("""
This part constructs nerve-immune-tumor communication networks from
co-expression patterns in TCGA data. 

Note: These are INFERRED networks based on gene co-expression.
Direct validation requires spatial technologies.

Outputs:
- Figure 6B: Nerve-immune co-expression network
- Figure 6D: Ligand-receptor signaling atlas
- Table S3: Network statistics
""")

# SECTION 10: COMPUTE CO-EXPRESSION NETWORKS

print("\n" + "="*80)
print("SECTION 10: LIGAND-RECEPTOR CO-EXPRESSION")
print("="*80)

print(f"\nComputing co-expression for ligand-receptor pairs...")

# Aggregate TCGA expression for network analysis
# Use top 3 cancers by nerve score for focused analysis
top_cancers = stats_df['cancer_type'].head(3).tolist()
print(f"\nFocusing on top 3 cancers: {', '.join(top_cancers)}")

lr_coexpression = {}

for cancer_type in top_cancers:
    print(f"\n{cancer_type}:")
    expr = tcga_expr_all[cancer_type]
    
    # Get all LR genes
    all_lr_genes = list(set(LR_PAIRS['ligand'].tolist() + LR_PAIRS['receptor'].tolist()))
    
    # Calculate co-expression
    corr_matrix, genes_present = calculate_correlation_network(expr, all_lr_genes, threshold=0.2)
    
    if corr_matrix is not None:
        lr_coexpression[cancer_type] = {
            'correlation_matrix': corr_matrix,
            'genes': genes_present
        }
        print(f"  [OK] Network: {len(genes_present)} genes, {(abs(corr_matrix.values)>0).sum()//2} edges")

# Save network data
network_file = OUTPUT_DIR / "ligand_receptor_networks.pkl"
import pickle
with open(network_file, 'wb') as f:
    pickle.dump(lr_coexpression, f)

print(f"\n[OK] Networks saved: {network_file.name}")

# SECTION 11: ANALYZE SIGNALING PATTERNS

print("\n" + "="*80)
print("SECTION 11: SIGNALING PATTERN ANALYSIS")
print("="*80)

# For each LR pair, compute co-expression strength across cancers
lr_signaling_strengths = []

for _, pair in LR_PAIRS.iterrows():
    ligand = pair['ligand']
    receptor = pair['receptor']
    
    for cancer_type in top_cancers:
        if cancer_type not in lr_coexpression:
            continue
        
        corr_matrix = lr_coexpression[cancer_type]['correlation_matrix']
        genes = lr_coexpression[cancer_type]['genes']
        
        if ligand in genes and receptor in genes:
            corr_value = corr_matrix.loc[ligand, receptor]
            
            lr_signaling_strengths.append({
                'cancer_type': cancer_type,
                'ligand': ligand,
                'receptor': receptor,
                'pathway': pair['pathway'],
                'direction': pair['direction'],
                'correlation': corr_value
            })

lr_signal_df = pd.DataFrame(lr_signaling_strengths)

if len(lr_signal_df) > 0:
    print(f"\n[OK] Analyzed {len(lr_signal_df)} LR interactions")
    
    # Summary by direction
    direction_summary = lr_signal_df.groupby('direction')['correlation'].agg(['mean', 'std', 'count'])
    print(f"\nSignaling by direction:")
    print(direction_summary)
    
    # Save
    signal_file = TABLE_DIR / "Table_S3_LR_Signaling_Strength.csv"
    lr_signal_df.to_csv(signal_file, index=False)
    print(f"\n[OK] Saved: {signal_file.name}")

# SECTION 12: CREATE FIGURE 6B/D - NETWORK DIAGRAMS

print("\n" + "="*80)
print("SECTION 12: CREATE FIGURE 6B & 6D - NETWORK VISUALIZATIONS")
print("="*80)

# Figure 6B: Heatmap of LR co-expression
if len(lr_signal_df) > 0:
    # Pivot to matrix
    lr_matrix = lr_signal_df.pivot_table(
        index='pathway',
        columns='cancer_type',
        values='correlation',
        aggfunc='mean'
    )
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    sns.heatmap(lr_matrix,
               cmap='RdBu_r',
               center=0,
               vmin=-0.5, vmax=0.5,
               annot=True,
               fmt='.2f',
               cbar_kws={'label': 'Co-expression (Spearman r)'},
               ax=ax)
    
    ax.set_title('Figure 6B: Ligand-Receptor Co-expression Across Cancer Types',
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Cancer Type', fontsize=10)
    ax.set_ylabel('Signaling Pathway', fontsize=10)
    
    plt.tight_layout()
    
    fig_file = FIG_DIR / "Figure_6B_LR_Coexpression.pdf"
    plt.savefig(fig_file, dpi=1200, bbox_inches='tight')
    plt.savefig(fig_file.with_suffix('.png'), dpi=1200, bbox_inches='tight')
    plt.close()
    
    print(f"\n[OK] Saved: {fig_file.name}")

# Figure 6D: Network diagram (simplified)
fig, ax = plt.subplots(figsize=(10, 8))

# Create simple conceptual network
cell_types = ['Nerves', 'Immune Cells', 'Tumor']
positions = {
    'Nerves': (0, 1),
    'Immune Cells': (1, 1),
    'Tumor': (0.5, 0)
}

# Draw nodes
for cell, (x, y) in positions.items():
    circle = plt.Circle((x, y), 0.15, color='lightblue', ec='black', linewidth=2, zorder=3)
    ax.add_patch(circle)
    ax.text(x, y, cell, ha='center', va='center', fontsize=10, fontweight='bold', zorder=4)

# Draw edges (arrows) representing signaling
arrows = [
    ('Nerves', 'Immune Cells', 'NGF/SP→'),
    ('Immune Cells', 'Nerves', '←IL-1β/TNF-α'),
    ('Tumor', 'Nerves', 'NGF/VEGF↑'),
    ('Tumor', 'Immune Cells', 'TGF-β↑')
]

for source, target, label in arrows:
    x1, y1 = positions[source]
    x2, y2 = positions[target]
    
    dx = x2 - x1
    dy = y2 - y1
    
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
               arrowprops=dict(arrowstyle='->', lw=2, color='gray'),
               zorder=2)
    
    # Label
    mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
    ax.text(mid_x, mid_y, label, fontsize=8, ha='center',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.set_xlim(-0.3, 1.3)
ax.set_ylim(-0.3, 1.3)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Figure 6D: Nerve-Immune-Tumor Communication Network',
            fontsize=12, fontweight='bold')

plt.tight_layout()

fig_file = FIG_DIR / "Figure_6D_Communication_Network.pdf"
plt.savefig(fig_file, dpi=1200, bbox_inches='tight')
plt.savefig(fig_file.with_suffix('.png'), dpi=1200, bbox_inches='tight')
plt.close()

print(f"\n[OK] Saved: {fig_file.name}")

# PART 3: PREDICTED NEUROPHYSIOLOGICAL OUTCOMES

print("\n" + "="*80)
print("="*80)
print("PART 3: PREDICTED NEUROPHYSIOLOGICAL OUTCOMES")
print("="*80)
print("="*80)

print("""
This part predicts neurophysiological and behavioral outcomes based on
nerve-immune gene expression patterns.

Outcomes analyzed:
- Pain (nociceptive markers)
- Fatigue (inflammatory cytokines)
- Depression (neurotransmitter systems)
- Cognition (cholinergic/dopaminergic)

Output: Figure 6C - Predicted outcomes by cancer type
""")

# SECTION 13: COMPUTE OUTCOME SCORES

print("\n" + "="*80)
print("SECTION 13: COMPUTE NEUROPHYSIOLOGICAL OUTCOME SCORES")
print("="*80)

outcome_scores_all = {}

for cancer_type in sorted(tcga_expr_all.keys()):
    print(f"\n{cancer_type}:")
    expr = tcga_expr_all[cancer_type]
    
    outcome_scores = {}
    
    for outcome, markers in NEURO_OUTCOMES.items():
        # Find overlapping markers
        overlap = list(set(expr.index) & set(markers))
        
        if len(overlap) == 0:
            outcome_scores[outcome] = pd.Series(index=expr.columns, data=np.nan)
            print(f"  {outcome}: 0 markers found")
            continue
        
        # Mean expression of markers
        expr_subset = expr.loc[overlap]
        scores = expr_subset.mean(axis=0)
        
        # Z-score normalize
        scores = (scores - scores.mean()) / (scores.std() + 1e-8)
        
        outcome_scores[outcome] = scores
        print(f"  {outcome}: {len(overlap)}/{len(markers)} markers, mean={scores.mean():.3f}")
    
    outcome_scores_all[cancer_type] = outcome_scores

# SECTION 14: CORRELATE OUTCOMES WITH NERVE SCORES

print("\n" + "="*80)
print("SECTION 14: NERVE SCORE vs OUTCOME CORRELATIONS")
print("="*80)

outcome_correlations = []

for cancer_type in sorted(tcga_scores.keys()):
    nerve_scores = tcga_scores[cancer_type]
    
    if cancer_type not in outcome_scores_all:
        continue
    
    outcomes = outcome_scores_all[cancer_type]
    
    for outcome_name, outcome_score in outcomes.items():
        # Match samples
        common_samples = list(set(nerve_scores.index) & set(outcome_score.index))
        
        if len(common_samples) < 10:
            continue
        
        x = nerve_scores[common_samples]
        y = outcome_score[common_samples]
        
        # Remove NaN
        mask = ~(np.isnan(x) | np.isnan(y))
        x = x[mask]
        y = y[mask]
        
        if len(x) < 10:
            continue
        
        corr, pval = stats.spearmanr(x, y)
        
        outcome_correlations.append({
            'cancer_type': cancer_type,
            'outcome': outcome_name,
            'n_samples': len(x),
            'correlation': corr,
            'p_value': pval,
            'significant': pval < 0.05
        })

outcome_corr_df = pd.DataFrame(outcome_correlations)

if len(outcome_corr_df) > 0:
    print(f"\n[OK] Computed {len(outcome_corr_df)} correlations")
    
    # Summary
    sig_corr = outcome_corr_df[outcome_corr_df['significant']]
    print(f"  Significant: {len(sig_corr)}/{len(outcome_corr_df)}")
    
    # Save
    corr_file = TABLE_DIR / "Table_S4_Outcome_Correlations.csv"
    outcome_corr_df.to_csv(corr_file, index=False)
    print(f"\n[OK] Saved: {corr_file.name}")

# SECTION 15: CREATE FIGURE 6C - PREDICTED OUTCOMES

print("\n" + "="*80)
print("SECTION 15: CREATE FIGURE 6C - PREDICTED OUTCOMES HEATMAP")
print("="*80)

if len(outcome_corr_df) > 0:
    # Pivot to matrix
    outcome_matrix = outcome_corr_df.pivot_table(
        index='outcome',
        columns='cancer_type',
        values='correlation',
        aggfunc='mean'
    )
    
    fig, ax = plt.subplots(figsize=(10, 5))
    
    sns.heatmap(outcome_matrix,
               cmap='RdBu_r',
               center=0,
               vmin=-0.5, vmax=0.5,
               annot=True,
               fmt='.2f',
               cbar_kws={'label': 'Correlation with Nerve Score'},
               ax=ax)
    
    ax.set_title('Figure 6C: Predicted Neurophysiological Outcomes by Cancer Type',
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Cancer Type', fontsize=10)
    ax.set_ylabel('Predicted Outcome', fontsize=10)
    
    # Add asterisks for significance
    for i, outcome in enumerate(outcome_matrix.index):
        for j, cancer in enumerate(outcome_matrix.columns):
            corr_data = outcome_corr_df[
                (outcome_corr_df['outcome']==outcome) & 
                (outcome_corr_df['cancer_type']==cancer)
            ]
            
            if len(corr_data) > 0 and corr_data.iloc[0]['p_value'] < 0.05:
                ax.text(j+0.5, i+0.5, '*', ha='center', va='center',
                       fontsize=16, fontweight='bold', color='black')
    
    plt.tight_layout()
    
    fig_file = FIG_DIR / "Figure_6C_Predicted_Outcomes.pdf"
    plt.savefig(fig_file, dpi=1200, bbox_inches='tight')
    plt.savefig(fig_file.with_suffix('.png'), dpi=1200, bbox_inches='tight')
    plt.close()
    
    print(f"\n[OK] Saved: {fig_file.name}")

# PART 4: THERAPEUTIC TARGET PRIORITIZATION

print("\n" + "="*80)
print("="*80)
print("PART 4: THERAPEUTIC TARGET PRIORITIZATION")
print("="*80)
print("="*80)

print("""
This part prioritizes therapeutic targets based on:
1. Expression levels across cancers
2. Co-expression with nerve injury programs
3. Known druggability
4. Predicted impact on nerve-immune crosstalk

Output: Figure 6E - Prioritized targets
""")

# SECTION 16: RANK THERAPEUTIC TARGETS

print("\n" + "="*80)
print("SECTION 16: THERAPEUTIC TARGET RANKING")
print("="*80)

# Define druggable targets (receptors primarily)
druggable_targets = [
    ('NTRK1', 'Neurotrophin signaling', 'Larotrectinib (approved)'),
    ('TACR1', 'Substance P signaling', 'Aprepitant (approved)'),
    ('IL1R1', 'IL-1 signaling', 'Anakinra (approved)'),
    ('IL6R', 'IL-6 signaling', 'Tocilizumab (approved)'),
    ('TNFRSF1A', 'TNF signaling', 'Etanercept (approved)'),
    ('TGFBR1', 'TGF-beta signaling', 'Galunisertib (clinical)'),
    ('CALCRL', 'CGRP signaling', 'Erenumab (approved)'),
    ('NPY1R', 'NPY signaling', 'Experimental')
]

target_scores = []

for target, pathway, drug in druggable_targets:
    print(f"\n{target} ({pathway}):")
    
    # Expression across cancers
    expr_levels = []
    corr_with_nerve = []
    
    for cancer_type in sorted(tcga_expr_all.keys()):
        expr = tcga_expr_all[cancer_type]
        
        if target not in expr.index:
            continue
        
        # Mean expression
        mean_expr = expr.loc[target].mean()
        expr_levels.append(mean_expr)
        
        # Correlation with nerve score
        nerve_scores = tcga_scores[cancer_type]
        common = list(set(expr.columns) & set(nerve_scores.index))
        
        if len(common) > 10:
            x = expr.loc[target, common]
            y = nerve_scores[common]
            
            mask = ~(np.isnan(x) | np.isnan(y))
            if mask.sum() > 10:
                corr, _ = stats.spearmanr(x[mask], y[mask])
                corr_with_nerve.append(abs(corr))
    
    if len(expr_levels) > 0:
        mean_expression = np.mean(expr_levels)
        mean_correlation = np.mean(corr_with_nerve) if corr_with_nerve else 0
        
        # Composite score (higher is better target)
        composite_score = mean_expression * mean_correlation
        
        target_scores.append({
            'target': target,
            'pathway': pathway,
            'drug_status': drug,
            'mean_expression': mean_expression,
            'mean_abs_correlation': mean_correlation,
            'composite_score': composite_score,
            'n_cancers': len(expr_levels)
        })
        
        print(f"  Expression: {mean_expression:.2f}")
        print(f"  |Correlation|: {mean_correlation:.3f}")
        print(f"  Score: {composite_score:.3f}")

target_df = pd.DataFrame(target_scores)
target_df = target_df.sort_values('composite_score', ascending=False)

print(f"\n[OK] Ranked {len(target_df)} targets")
print(f"\nTop 5 targets:")
print(target_df[['target', 'pathway', 'drug_status', 'composite_score']].head())

# Save
target_file = TABLE_DIR / "Table_S5_Therapeutic_Targets_Ranked.csv"
target_df.to_csv(target_file, index=False)
print(f"\n[OK] Saved: {target_file.name}")

# SECTION 17: CREATE FIGURE 6E - TARGET PRIORITIZATION

print("\n" + "="*80)
print("SECTION 17: CREATE FIGURE 6E - THERAPEUTIC TARGETS")
print("="*80)

if len(target_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Panel 1: Bar chart of composite scores
    ax = axes[0]
    
    y_pos = np.arange(len(target_df))
    ax.barh(y_pos, target_df['composite_score'], color='#fdb462', alpha=0.8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(target_df['target'])
    ax.set_xlabel('Target Priority Score', fontsize=10)
    ax.set_title('Therapeutic Target Ranking', fontsize=11, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Panel 2: Expression vs Correlation scatter
    ax = axes[1]
    
    colors = ['green' if 'approved' in status.lower() else 'orange' 
             for status in target_df['drug_status']]
    
    ax.scatter(target_df['mean_expression'],
              target_df['mean_abs_correlation'],
              s=100, c=colors, alpha=0.6, edgecolors='black')
    
    # Labels
    for _, row in target_df.iterrows():
        ax.annotate(row['target'], 
                   (row['mean_expression'], row['mean_abs_correlation']),
                   fontsize=8, ha='right')
    
    ax.set_xlabel('Mean Expression Across Cancers', fontsize=10)
    ax.set_ylabel('|Correlation with Nerve Score|', fontsize=10)
    ax.set_title('Target Expression vs Nerve Association', fontsize=11, fontweight='bold')
    ax.grid(alpha=0.3)
    
    # Legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='green', label='Approved drugs'),
        Patch(facecolor='orange', label='Clinical/Experimental')
    ]
    ax.legend(handles=legend_elements, loc='best')
    
    plt.suptitle('Figure 6E: Prioritized Therapeutic Targets',
                fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    
    fig_file = FIG_DIR / "Figure_6E_Therapeutic_Targets.pdf"
    plt.savefig(fig_file, dpi=1200, bbox_inches='tight')
    plt.savefig(fig_file.with_suffix('.png'), dpi=1200, bbox_inches='tight')
    plt.close()
    
    print(f"\n[OK] Saved: {fig_file.name}")


# FINAL SUMMARY


print("\n" + "="*80)
print("="*80)
print("NOTEBOOK 4 COMPLETE")
print("="*80)
print("="*80)

print(f"\nEnd time: {datetime.now()}")

print(f"\n{'='*80}")
print("OUTPUTS GENERATED")
print(f"{'='*80}\n")

print("Main Figures:")
print("  [OK] Figure 6A: Pan-cancer nerve score distribution")
print("  [OK] Figure 6B: Ligand-receptor co-expression")
print("  [OK] Figure 6C: Predicted neurophysiological outcomes")
print("  [OK] Figure 6D: Communication network diagram")
print("  [OK] Figure 6E: Therapeutic target prioritization")

print("\nSupplementary Tables:")
print("  [OK] Table S1: TCGA nerve score statistics")
print("  [OK] Table S2: Nerve scores vs immune phenotypes")
print("  [OK] Table S3: Ligand-receptor signaling strength")
print("  [OK] Table S4: Outcome correlations")
print("  [OK] Table S5: Therapeutic targets ranked")

print(f"\n{'='*80}")
print("MANUSCRIPT INTEGRATION")
print(f"{'='*80}\n")

print("""
NB4 provides:
1. ✓ Pan-cancer exploratory context (TCGA - 15 cancer types)
2. ✓ Inferred nerve-immune-tumor communication networks
3. ✓ Predicted neurophysiological outcomes (pain, fatigue, etc.)
4. ✓ Prioritized therapeutic targets with druggability
5. ✓ Perfect alignment with special issue theme

Special Issue Fit:
- "Cancer and Immune Interactions" ✓
- "Implications for Neurophysiology and Behavior" ✓
- "Provide roadmap for the field" ✓

Key Messages for Manuscript:
- TCGA analyses are exploratory (prevalence mapping)
- Network inference from co-expression (hypothesis-generating)
- Spatial validation (NB3) is primary evidence
- First comprehensive nerve-immune-tumor atlas
- Actionable therapeutic targets identified
""")

print(f"{'='*80}")
print("NEXT STEPS")
print(f"{'='*80}\n")

print("""
1. Review all generated figures (Figure 6A-E)
2. Check supplementary tables (Table S1-S5)
3. Update manuscript with NB4 results
4. Emphasize exploratory nature of TCGA
5. Highlight therapeutic predictions (special issue fit!)
6. Write figure legends
7. Finalize for submission

Ready for submission to Neuroimmunomodulation special issue! 🎯
""")

print(f"{'='*80}\n")
print("END OF NOTEBOOK 4")
print(f"{'='*80}\n")